In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import h5py

%matplotlib widget
from matplotlib import pyplot as plt
# plt.style.use('fivethirtyeight')

from mpi4py import MPI

import numpy as np

from pyMoBiMP.gmsh_utils import dfx_spherical_mesh

from pyMoBiMP.plotting_utils import (
    add_arrow,
    plot_charging_cycle,
    plot_time_sequence,
    PyvistaAnimation,
    animate_time_series)

comm_world = MPI.COMM_WORLD

## Read in the data

In [ ]:
filename = "../simulation_output/CH_4_min_2_particle.h5"

with h5py.File(filename, 'r') as f:
    print(f["Function"].keys())

    # grid coordinates
    x_data = f["Mesh/mesh/geometry"][()]

    t_keys = f["Function/y_0"].keys()

    # time steps (convert from string to float)
    t = [float(t.replace("_", ".")) for t in t_keys]

    # list of data stored as numpy arrays
    u_data_1 = np.array([
        (f["Function/y_0"][u_key][()].squeeze(),
         f["Function/mu_0"][u_key][()].squeeze())
        for u_key in t_keys])

    u_data_2 = np.array([
        (f["Function/y_1"][u_key][()].squeeze(),
         f["Function/mu_1"][u_key][()].squeeze())
        for u_key in t_keys])

sorted_indx = np.argsort(t)

t = np.array(t)[sorted_indx]
u_data_1 = np.array(u_data_1)[sorted_indx]
u_data_2 = np.array(u_data_2)[sorted_indx]

rt_data  = np.loadtxt("../simulation_output/CH_4_min_2_particle_rt.txt")

In [ ]:
fig, ax = plot_time_sequence((x_data, t, u_data_1), lambda y: np.exp(y) / (1 + np.exp(y)))

plt.show()

In [ ]:
fig, ax = plot_time_sequence((x_data, t, u_data_2), lambda y: np.exp(y) / (1 + np.exp(y)))

plt.show()

In [ ]:
mu_1_bc = u_data_1[:, 1, -1]
mu_2_bc = u_data_2[:, 1, -1]

mu_diff = mu_1_bc - mu_2_bc

plt.figure()

plt.plot(t, mu_diff)

plt.show()

print(np.abs(mu_1_bc).max())
print(np.abs(mu_2_bc).max())
print(np.abs(u_data_1[:, 1, -1] - u_data_2[:, 1, -1]).max())

In [ ]:
np.abs(u_data_2 - u_data_1).max()

In [ ]:
mesh_3d, _, _ = dfx_spherical_mesh(comm_world, resolution=0.1, optimize=False)

In [ ]:
a = 1.5
b = 0.2
cc = 5.

chem_pot = lambda q: np.log(q / (1 - q)) + a * (1 - 2 * q) + b * np.pi * cc * np.cos(np.pi * cc * q)

anim = PyvistaAnimation(
    (x_data, t, u_data_1),
    rt_data=rt_data,
    mesh_3d=mesh_3d,
    c_of_y=lambda y: np.exp(y) / (1 + np.exp(y)),
    f_of_q=chem_pot,
    res=1.0,
    clim=[0.0, 1.0],
    cmap="fire",
)

widget = anim.get_slider_widget()